In [4]:
# chall 1

from base64 import b64encode 

dat = bytearray.fromhex("49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d")
print(b64encode(dat))

b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'


In [35]:
# chall 2

def xor(a, b):
  ret = []
  for ab, bb in zip(a, b):
    ret.append(ab^bb)
  return bytearray(ret) 

a = bytearray.fromhex("1c0111001f010100061a024b53535009181c")
b = bytearray.fromhex("686974207468652062756c6c277320657965")
print(xor(a, b).hex())

746865206b696420646f6e277420706c6179


In [55]:
# chall 3

f = {" ": 15, "e": 12.02, "t": 9.10, "a": 8.12, "o": 7.68, "i": 7.31, "n": 6.95, "s": 6.28, "r": 6.02, "h": 5.92, "d": 4.32, "l": 3.98, "u": 2.88, "c": 2.71, "m": 2.61, "f": 2.30, "y": 2.11, "w": 2.09, "g": 2.03, "p": 1.82, "b": 1.49, "v": 1.11, "k": 0.69, "x": 0.17, "q": 0.11, "j": 0.10, "z": 0.07}
  
# how englishy is a string?
def score(a):
  return sum([f.get(chr(c).lower(), 0) for c in a])

def unxor(a):
  ret = []
  key = ""
  for k in range(0, 255):
    b = [k ^ c for c in a]
    if score(b) > score(ret): ret, key = b, k
  return bytearray(ret), key

a = bytearray.fromhex("1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736")
print(unxor(a)[0].decode())

Cooking MC's like a pound of bacon


In [56]:

# chall 4

# use score and unxor from solution for #3 

# use frequency scoring to find most englishy result
def detect_xor(a):
  ret, ret_score = "", 0
  for line in a:
    s = score(unxor(line)[0])
    if  s > ret_score:
      ret = unxor(line)[0]
      ret_score = s
  return ret

dat = [bytearray.fromhex(l.strip()) for l in open("4.txt").readlines()]
print(detect_xor(dat).decode())

Now that the party is jumping



In [31]:
# chall 5

from binascii import hexlify

def xor(a, key):
  ret = list()
  for i, c in enumerate(a):
    ret.append(c ^ key[i % len(key)])
  return bytearray(ret)

plain = bytearray([ord(c) for c in open("5.txt").read().strip()])
key = bytearray([ord(c) for c in "ICE"])
print(hexlify(xor(plain, key)))

b'0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'


In [34]:
# chall 6

from base64 import b64decode
from itertools import permutations

# get hamming distance
def hamming(a, b):
  ret = 0
  for i, j in zip(a, b):
    ret += sum(int(n) for n in bin(i^j)[2:])
  return ret + abs(len(a)-len(b))

# break repeating key xor
def xor_smash(a):
  ks, dmin =  None, None
  for s in range(2, 50):
    b = list(permutations([a[n*s:n*s+s] for n in range(4)], 2))
    d = sum([hamming(i, j) for i, j in b]) / (len(b)*s)
    if ks is None or d < dmin: ks, dmin = s, d

  blocks = [list() for _ in range(ks)]
  for b in [a[i:i+ks] for i in range(0, len(a), ks)]:
    for j, bb in enumerate(b):
      blocks[j].append(bb)

  key = [unxor(c)[1] for c in blocks]
  return xor(a, key), key

enc = b64decode(open("6.txt").read())
print(xor_smash(enc)[0].decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


In [38]:
# chall 7

from base64 import b64decode
from Crypto.Cipher import AES

def aes128_decrypt(cipher, key):
  c = AES.new(key, AES.MODE_ECB)
  return c.decrypt(cipher)

cipher = b64decode(open("7.txt").read())
key = "YELLOW SUBMARINE"
print(aes128_decrypt(cipher, key))

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

In [41]:
# chall 8

# how ECB-y are a set of bytes?
# higher is better
def ecb_score(a):
  # break into 16 byte chunks
  a = bytes(a)
  c = dict()
  for b in range(0, len(a), 16):
    c[a[b:b+16]] = c.get(a[b:b+16], 0) + 1

  #if max(c.values()) > 2: print(c.items())
  return max(c.values())

def detect_ecb(a):
  xx = list()
  ret, vmax = None, 0
  for l in a:
    xx.append(ecb_score(l))
    if ret is None or ecb_score(l) > vmax: ret = l, ecb_score(l)
  return l

cipher = [bytearray.fromhex(l.strip()) for l in open("8.txt").readlines()]
print(detect_ecb(cipher))

bytearray(b"\x06\xdf\x04\x18\x882\xb1\n\xff\xf9B\t\xd2\xaa\x1c\x8a\x127\x02\xde(#M\xcd>\n}6\xc1\xaa\x84I\xe6\xfaU\xe3\xe1\xe3\xd7}\x84$\xe8zE\xe3\x86\x97u_\x84\xc4\x9a\x99G7\x97&\x81\x13\xebi\t\x88\x88\x94u&\x03[$m\x00\xa60\xf6 \x1e\xcc@u\xd8\xaaf\x04\xdes\xe2\x11\x9e&NL\x96u\x1f*g\xa2\xe4l\xf4g\xa0\xdf\x8f\x05 \xbc\xf4v+\'\x15\xab\xa2f\xd9\xb3\xf5\xe8\xfag\xd1/\x9c\xaa\xc9(\xb0z\xc3\xbe\x99\xf4\x11 eZ\xa7\x7fd3\xfc&Fs\xa9))\xe7\x92\x18\x7f\x87\xb5\xfd\xa5\x0c\xf2")
